In [1]:
#!pip install lm-eval==0.3.0 -qqq
!git clone https://github.com/EleutherAI/lm-evaluation-harness
!pip install -e ./lm-evaluation-harness/. -qqq

fatal: destination path 'lm-evaluation-harness' already exists and is not an empty directory.


In [2]:
import ctranslate2
import glob
import os
import pandas as pd
import peft
import random
import timeit
import urllib
import torch
import json
import os
from lm_eval import tasks, evaluator, utils
import lm_eval
import wandb
from peft import get_peft_model, LoraConfig, TaskType
from peft import PeftModel, PeftConfig
import wandb.apis.reports as wr
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Thanks for trying out the Report API!
wandb: For a tutorial, check out https://colab.research.google.com/drive/1CzyJx1nuOS4pdkXa2XPaRQyZdmFmLmXV
wandb: 
wandb: Try out tab completion to see what's available.
wandb:   ∟ everything:    `wr.<tab>`
wandb:       ∟ panels:    `wr.panels.<tab>`
wandb:       ∟ blocks:    `wr.blocks.<tab>`
wandb:       ∟ helpers:   `wr.helpers.<tab>`
wandb:       ∟ templates: `wr.templates.<tab>`
wandb:       
wandb: For bugs/feature requests, please create an issue on github: https://github.com/wandb/wandb/issues


In [3]:
os.environ["WANDB_PROJECT"] = "Autocompletion with evaluation"
os.environ["WANDB_ENTITY"] = "reviewco"
os.environ["WANDB_USERNAME"] = "keisuke-kamata"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"
os.environ["WANDB_WATCH"] = "gradients"

SCORE_TABLE_NAME = "Score"
EVALUATION_TABLE_NAME = "Validation Responses"
LATENCY_TABLE_NAME = "Model Latencies"
MODEL_NAME = "Instruction-tuned-model"

In [4]:
run = wandb.init(job_type="evaluation")
artifact = run.use_artifact('reviewco/Autocompletion with evaluation/finetuned-model:v1', type='model')
artifact_dir = artifact.download()    
base_llm = AutoModelForCausalLM.from_pretrained("facebook/opt-125m", device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m", local_files_only=True)
model = PeftModel.from_pretrained(base_llm, artifact_dir,torch_dtype=torch.float16)
model = model.merge_and_unload()

wandb: Currently logged in as: keisuke-kamata (reviewco). Use `wandb login --relogin` to force relogin


wandb:   3 of 3 files downloaded.  


In [5]:
# 評価を実行
results = lm_eval.evaluator.simple_evaluate(
    model=model,
    tasks=["arc_easy","hellaswag","squad2"],  
    batch_size=16,
    num_fewshot=3,
    device="cuda"  
)

Generating validation split: 100%|██████████| 11873/11873 [00:00<00:00, 51170.07 examples/s]


Task: arc_easy; number of docs: 2376
Task: arc_easy; document 0; context prompt (starting on next line):
Question: Homes that are built to be environmentally friendly because they use energy more efficiently than other homes are called "green" homes. "Green" homes often have reflective roofs and walls made of recycled materials. The windows in these energy-saving homes are double-paned, meaning each window has two pieces of glass. Double-paned windows have a layer of air between the window panes. This layer is a barrier against extreme temperatures and saves energy. A solar panel on a "green" home uses
Answer: a renewable energy source

Question: What is an environmental threat posed by oil and gas acquisition and transport that is not posed by the acquisition and transport of the energy resources uranium, and wood?
Answer: pipeline leaks

Question: One cause of heat in a car engine is caused by friction between parts. Which of these reduces the amount of heat produced by friction?
Ans

100%|██████████| 61542/61542 [12:18<00:00, 83.37it/s] 


Running greedy_until requests


100%|██████████| 11873/11873 [39:39<00:00,  4.99it/s] 
/work/lm-evaluation-harness/lm_eval/tasks/squad.py:36: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = datasets.load_metric("squad_v2")


In [17]:
results["results"]["squad2"]["f1"]

5.770005969375321

In [ ]:
table_contents = []
table_contents.append(run.id)
table_contents.append(results["results"]["arc_easy"]["acc"])
table_contents.append(results["results"]["hellaswag"]["acc"])
table_contents.append(results["results"]["hellaswag"]["acc"])
table = wandb.Table(columns=['finetuned-model:v1']+tasks,
                            data=[table_contents])
run.log({'result_table':table})
run.log_code()

In [ ]:
def get_completion_responses_batch(prompts, ft_path):
    # Get completions for each model in batches
    opt_completions = get_huggingface_completion_batch(prompts, "facebook/opt-125m")
    ft_completions = get_huggingface_completion_batch(prompts, ft_path)
    responses = []
    for opt, ft in zip(opt_completions, ft_completions):
        responses.append({
            "Production": opt,
            "Staging (finetuned)": ft
        })
    return responses

def get_huggingface_completion_batch(prompts, model):
    generator = pipeline('text-generation', model=model)
    responses = generator(prompts, max_new_tokens=50)
    completions = []
    for i, prompt in enumerate(prompts):
        full_output = responses[i][0]["generated_text"]
        output = full_output[len(prompt):] if full_output.startswith(prompt) else full_output
        completions.append(output.strip())
    return completions

def get_model_comparison_df(prompts, ft_path):
    trimmed_prompts = [
        " ".join(prompt.split()[:random.randint(5,12)])
        for prompt in prompts
    ]
    responses = get_completion_responses_batch(trimmed_prompts, ft_path)
    df = pd.DataFrame(responses)
    df.insert(0, "prompt", trimmed_prompts)
    return df

def get_latency_df(prompts, num_prompts, ft_path, ct2_path):
  prompts = prompts.to_list()[:num_prompts]
  ft_time = timeit.timeit(lambda: get_huggingface_completion_batch(prompts, model), number=1)
  opt_time = timeit.timeit(lambda: get_huggingface_completion_batch(prompts, "facebook/opt-125m"), number=1)

  return pd.DataFrame({
    "Model": ["Production", "Staging (finetuned)"],
    f"Latency for {num_prompts} Reviews": [opt_time, ft_time],
  })


In [ ]:


print(results)

In [ ]:
results

In [ ]:
with wandb.init(job_type="model_evaluation") as run:
    staging_model = wandb.use_artifact(f'{os.environ["WANDB_ENTITY"]}/model-registry/{MODEL_NAME}:staging')
    staging_path = staging_model.download()
    

    reviews_artifact = run.use_artifact(f'{os.environ["WANDB_ENTITY"]}/{os.environ["WANDB_PROJECT"]}/reviews:production')
    reviews_dir = reviews_artifact.download()

    test_files = glob.glob(f"{reviews_dir}/test/*.parquet")
    test_data = pd.concat([pd.read_parquet(path) for path in test_files])
    prompts = test_data.sample(frac=1)["text"][:10]

    wandb.log({
        EVALUATION_TABLE_NAME: get_model_comparison_df(prompts, ft_path=staging_path),
        LATENCY_TABLE_NAME: get_latency_df(prompts, num_prompts=3, ft_path=staging_path)
    })


In [ ]:

# Report
    report = wr.Report(
        project=os.environ["WANDB_PROJECT"],
        entity=os.environ["WANDB_ENTITY"],
        title='Model Evaluation: Autocompletion Model',
        description="Data and sample predictions to evaluate the staging candidate model for our review autocompletion algorithm."
    )

    report.width = "fluid"

    runsets = [wr.Runset(
        os.environ["WANDB_ENTITY"],
        os.environ['WANDB_PROJECT']
        )]

    report.blocks = [
        wr.TableOfContents(),
        wr.H1("Report Overview"),
        wr.P(
            "This report contains information to evaluate whether potential staging models "
            "should be moved to production. Model Registry admins can use the view of the "
            "Model Registry at the end of this report to move a staging model into production, "
            "using a Webhook automation."
        ),
        wr.Spotify(spotify_id="7KAveXwQ5xzdHT6GDlNIBu"),
        wr.MarkdownBlock("May this staging model earn 5 stars 🙏."),
        wr.HorizontalRule(),
    ]

    pg = wr.PanelGrid(
        runsets=runsets,
        panels=[
        wr.ScalarChart(
            title="Current Min Eval Loss",
            metric="eval/loss",
            groupby_aggfunc="min",
            font_size="large"),

        wr.ScalarChart(
            title="Current Min Train Loss",
            metric="train/loss",
            groupby_aggfunc="min",
            font_size="large"),

        wr.ScalarChart(
            title="Longest Runtime (sec)",
            metric="train/train_runtime",
            groupby_aggfunc="max",
            font_size="large"),

        wr.LinePlot(x='Step',
                    y=['eval/loss'],
                    smoothing_factor=0.8,
                    layout={'w': 24, 'h': 9})
        ]
    )

    report.blocks += [wr.H1("Key Metrics"), pg]

    pg = wr.PanelGrid(
        runsets=runsets,
        panels=[
            wr.WeavePanelSummaryTable(LATENCY_TABLE_NAME, layout={'w': 24, 'h': 12}),
        ])

    report.blocks += [wr.H1("Latency Data for Models"), pg]


    pg = wr.PanelGrid(
        runsets=runsets,
        panels=[
            wr.WeavePanelSummaryTable(EVALUATION_TABLE_NAME, layout={'w': 24, 'h': 12}),
        ])

    report.blocks += [wr.H1("Sample Predictions"), pg]

    report.blocks += [wr.H1("Autocompletion Model in Model Registry"), wr.WeaveBlockArtifact(os.environ["WANDB_ENTITY"], "model-registry", MODEL_NAME)]
    report.save()

    report_creation_msg = f"Report to review: {urllib.parse.quote(report.url, safe=r'/:')}"
    print(report_creation_msg)

    wandb.alert("New Staging Model Evaluated", report_creation_msg)